In [1]:
# her skal jeg lage noe
# Import the necessary library
from ortools.linear_solver import pywraplp
import random

# Initialize the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Define constants and parameters
L = 3  # Number of levels (you can adjust as needed)

# Generate random data for constants
C_I = random.randint(100, 200)  # Cost to establish the third park
N = random.randint(50, 100)     # Maximum number of units per park
D = random.randint(200, 300)    # Total demand

# Generate random data for P_i, Q_l, C^M_i, and C_{i\ell}^L
P = [random.randint(5, 10) for _ in range(3)]  # P_i
Q = [random.randint(80, 120) for _ in range(L)]  # Q_l

C_M = [random.randint(20, 40) for _ in range(3)]  # C^M_i
C_L = [[random.randint(10, 20) for _ in range(L)] for _ in range(3)]  # C_{i\ell}^L

# Decision variables
x = [solver.IntVar(0, N, f'x_{i+1}') for i in range(3)]  # x_i variables
delta = [[solver.BoolVar(f'delta_{i+1}_{l+1}') for l in range(L)] for i in range(3)]  # delta_{i\ell} variables
y = solver.BoolVar('y')  # y variable

# Objective function
objective_terms = []
# Cost of manufacturing
objective_terms.extend(C_M[i] * x[i] for i in range(3))
# Cost related to delta variables
objective_terms.extend(C_L[i][l] * delta[i][l] for i in range(3) for l in range(L))
# Cost of establishing the third park
objective_terms.append(C_I * y)

solver.Minimize(solver.Sum(objective_terms))

# Constraints

# Constraint 1: Total production meets demand
solver.Add(solver.Sum(P[i] * x[i] for i in range(3)) >= D)

# Constraint 2: x_i <= N (already defined in variable bounds)

# Constraint 3: Production limits based on delta variables
for i in range(3):
    solver.Add(P[i] * x[i] <= solver.Sum(Q[l] * delta[i][l] for l in range(L)))

# Constraint 4: Each park must choose exactly one level
for i in range(3):
    solver.Add(solver.Sum(delta[i][l] for l in range(L)) == 1)

# Constraint 5: x_3 <= N * y
solver.Add(x[2] <= N * y)

# Constraint 6: Additional production constraint involving parks 1 and 3
solver.Add(P[0] * x[0] + P[2] * x[2] <= solver.Sum(Q[l] * (delta[0][l] + delta[2][l]) for l in range(L)))

# Solve the model
status = solver.Solve()

# Output the results
if status == pywraplp.Solver.OPTIMAL:
    print('Optimal solution found:')
    print(f'Objective value (z) = {solver.Objective().Value()}')
    for i in range(3):
        print(f'x_{i+1} = {x[i].solution_value()}')
    print(f'y = {y.solution_value()}')
    for i in range(3):
        for l in range(L):
            print(f'delta_{i+1}_{l+1} = {delta[i][l].solution_value()}')
else:
    print('No optimal solution found.')

# Print the data used for transparency
print('\nData used:')
print(f'C_I (Cost to establish third park) = {C_I}')
print(f'N (Max units per park) = {N}')
print(f'D (Total demand) = {D}')
print(f'P_i (Production coefficients) = {P}')
print(f'Q_l (Quantities for levels) = {Q}')
print(f'C_M_i (Manufacturing costs) = {C_M}')
print('C_L_i_l (Level costs):')
for i in range(3):
    print(f'  Park {i+1}: {C_L[i]}')


Optimal solution found:
Objective value (z) = 1391.0
x_1 = 4.0
x_2 = 13.0
x_3 = 24.0
y = 1.0
delta_1_1 = -0.0
delta_1_2 = 1.0
delta_1_3 = -0.0
delta_2_1 = -0.0
delta_2_2 = 1.0
delta_2_3 = 0.0
delta_3_1 = -0.0
delta_3_2 = 1.0
delta_3_3 = 0.0

Data used:
C_I (Cost to establish third park) = 122
N (Max units per park) = 89
D (Total demand) = 254
P_i (Production coefficients) = [5, 9, 5]
Q_l (Quantities for levels) = [87, 120, 83]
C_M_i (Manufacturing costs) = [39, 29, 29]
C_L_i_l (Level costs):
  Park 1: [18, 14, 17]
  Park 2: [19, 11, 10]
  Park 3: [16, 15, 14]
